In [1]:
import numpy as np
from numpy.ma import masked_values as maval
import gsw
import xarray as xr
import pandas as pd
import os.path as op
from datetime import datetime, timedelta
from scipy.interpolate import PchipInterpolator as pchip
from scipy.interpolate import interp1d
from scipy.signal import medfilt
import scipy.stats as stats
import dask.array as dsar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import cm
import matplotlib.colors as clr
import matplotlib.ticker as tick
import matplotlib.path as mpath
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ddir = '/swot/SUM02/takaya/SOCCOM/SIO'

In [3]:
pf = pd.read_csv(op.join(ddir,'../Fronts/pf.txt'), delim_whitespace=True, 
                 nrows=968, header=None, dtype=float, names=['lon','lat'])
saf = pd.read_csv(op.join(ddir,'../Fronts/saf.txt'), delim_whitespace=True, 
                 nrows=1079, header=None, dtype=float, names=['lon','lat'])

In [6]:
pnew = np.append(np.arange(4,100,2),np.arange(100,310,10))
dp = np.diff(pnew)[:24]
dp = np.append(dp, 7)
dp = np.append(dp, np.diff(pnew[24:]))

v = 0
k,l,m,q = (0,0,0,0)
for floatid in range(4180,5986):
    fname = op.join(ddir,'590'+str(floatid)+'_Mprof.nc')
    if op.exists(fname):
        ds = xr.open_dataset(fname)
        if 'BBP700' in ds and ~np.isnan(ds.BBP700.data).all() and len(ds.N_PROF)>2:
            lat = ds.LATITUDE
            lon = ds.LONGITUDE
            p = ds.PRES_ADJUSTED.where(xr.DataArray(maval(maval(maval(maval(maval(np.asarray(ds.PRES_ADJUSTED_QC,
                                                                                             dtype=float),
                                                                                 0.),3.),4.),6.),7.), 
                                                    dims=ds.PRES_ADJUSTED_QC.dims, coords=ds.PRES_ADJUSTED_QC.coords)<9.)
            p = np.ma.masked_invalid(p.where(p!=99999.).where(p>0.))
            maskp = p.mask

            t = ds.TEMP_ADJUSTED.where(xr.DataArray(maval(maval(maval(maval(maval(np.asarray(ds.TEMP_ADJUSTED_QC,
                                                                                             dtype=float),
                                                                                 0.),3.),4.),6.),7.), 
                                                    dims=ds.TEMP_ADJUSTED_QC.dims, coords=ds.TEMP_ADJUSTED_QC.coords)<9.)
            t = t.where(t!=99999.)
            s = ds.PSAL_ADJUSTED.where(xr.DataArray(maval(maval(maval(maval(maval(np.asarray(ds.PSAL_ADJUSTED_QC,
                                                                                             dtype=float),
                                                                                 0.),3.),4.),6.),7.), 
                                                    dims=ds.PSAL_ADJUSTED_QC.dims, coords=ds.PSAL_ADJUSTED_QC.coords)<9.)
            s = s.where(s!=99999.).where(s>0.)
            SA = xr.apply_ufunc(gsw.SA_from_SP,s,p,lon,lat)
            pt = xr.apply_ufunc(gsw.pt0_from_t,SA,t,p)
            CT = xr.apply_ufunc(gsw.CT_from_pt,SA,pt)
            rho = np.ma.masked_invalid(xr.apply_ufunc(gsw.rho,SA,CT,0))
            maskr = rho.mask

            bp700 = ds.BBP700.where(xr.DataArray(maval(maval(maval(maval(np.asarray(ds.BBP700_QC,dtype=float),
                                                                        0.),4.),6.),7.), 
                                                dims=ds.BBP700_QC.dims, coords=ds.BBP700_QC.coords)<9.
                                     ).where(ds.BBP700!=99999.)
            chl = ds.CHLA_ADJUSTED.where(xr.DataArray(maval(maval(maval(maval(maval(np.asarray(ds.CHLA_ADJUSTED_QC,
                                                                                               dtype=float),
                                                                                    0.),3.),4.),6.),7.), 
                                                    dims=ds.CHLA_ADJUSTED_QC.dims, coords=ds.CHLA_ADJUSTED_QC.coords)<9.
                                          ).where(ds.CHLA_ADJUSTED!=99999.)
            if np.nanmax(p)>6e2:
                chl -= np.nanmedian(chl.where(p>=600))
                bp700 -= np.nanmedian(bp700.where(p>=600))
            bp700 = bp700.where(bp700>0.)
            chl = chl.where(chl>0.)
            bpmed = bp700.copy()
            chmed = chl.copy()
            for i in range(len(ds.N_PROF)):
                bpmed[i] = medfilt(bp700[i], 5)
                chmed[i] = medfilt(chl[i], 5)

            bp470 = bpmed * (470/700)**-.78
            Cphyto = np.ma.masked_invalid(12128*bp470 + .59)
            maskC = Cphyto.mask 
            bpmed = np.ma.masked_array(bpmed, mask=maskC)
            chmed = np.ma.masked_invalid(chmed)
            maskc = chmed.mask

            pMLD = np.empty(len(ds.N_PROF))
            Chl = np.empty((len(ds.N_PROF),len(pnew)))
            Rho = np.empty((len(ds.N_PROF),len(pnew)))
            C = np.empty((len(ds.N_PROF),len(pnew)))
            bp = np.empty((len(ds.N_PROF),len(pnew)))

            for i in range(len(ds.N_PROF)):
                if len(chmed[i].compressed()) > 0. and all(np.diff(np.ma.masked_array(p,maskc)[i].compressed())>0.):
                    func1 = pchip(np.ma.masked_array(p,maskc)[i].compressed(),
                                 np.ma.masked_array(chmed,maskp)[i].compressed(),extrapolate=False)
                    Chl[i] = func1(pnew)
                else:
                    Chl[i] = np.ones(len(pnew))*np.nan

                if len(rho[i].compressed()) > 2 and all(np.diff(np.ma.masked_array(p,maskr)[i].compressed())>0.):
                    for j in range(len(ds.N_LEVELS)):
                        if ~np.isnan(rho.data[i,j]):
                            top = j
                            break
                    if p[i,top]<=2e1:
                        itop = int(np.nanargmin(np.abs(p[i]-10.)))
                        pMLD[i] = p[i,itop+np.nanargmin(np.abs((rho[i,itop:]-rho[i,itop])-0.03))]
                    else:
                        pMLD[i] = np.nan
                    func2 = pchip(np.ma.masked_array(p,maskr)[i].compressed(),
                                 np.ma.masked_array(rho,maskp)[i].compressed(),extrapolate=False)
                    Rho[i] = func2(pnew)
                else:
                    pMLD[i] = np.nan
                    Rho[i] = np.ones(len(pnew))*np.nan

                if len(Cphyto[i].compressed()) > 0. and all(np.diff(np.ma.masked_array(p,maskC)[i].compressed())>0.):
                    func3 = pchip(np.ma.masked_array(p,maskC)[i].compressed(),
                                 np.ma.masked_array(Cphyto,maskp)[i].compressed(),extrapolate=False)
                    C[i] = func3(pnew)
                else:
                    C[i] = np.ones(len(pnew))*np.nan

                if len(bpmed[i].compressed()) > 0. and all(np.diff(np.ma.masked_array(p,maskC)[i].compressed())>0.):
                    func4 = pchip(np.ma.masked_array(p,maskC)[i].compressed(),
                                 np.ma.masked_array(bpmed,maskp)[i].compressed(),extrapolate=False)
                    bp[i] = func4(pnew)
                else:
                    bp[i] = np.ones(len(pnew))*np.nan

            dC = np.diff(C,axis=-1)/np.diff(pnew)[np.newaxis,:]
            for i in range(C.shape[0]):
                pcrit = np.nanmax(np.array([1e2,pMLD[i]]))
                for j in range(C.shape[1]-1):
                    if pnew[j] > pcrit:
                        if dC[i,j] > 1.:
                            C[i,j+1] = np.nan
                        elif dC[i,j] < -1.:
                            C[i,j] = np.nan
            C = np.ma.masked_invalid(C)
            maskC = C.mask
            bp = np.ma.masked_array(bp, maskC)
            
            Chl = xr.DataArray(Chl, dims=['time','pres'],
                              coords={'time':np.asarray(ds.JULD,dtype='datetime64[s]'),'pres':pnew,
                                     'lat':('time',lat),'lon':('time',lon)}
                              ).dropna('time', how='all')
            C = xr.DataArray(C, dims=['time','pres'], 
                            coords={'time':np.asarray(ds.JULD,dtype='datetime64[s]'),'pres':pnew,
                                   'lat':('time',lat),'lon':('time',lon)}
                            ).dropna('time', how='all')
            bp = xr.DataArray(bp, dims=['time','pres'], 
                             coords={'time':np.asarray(ds.JULD,dtype='datetime64[s]'),'pres':pnew,
                                    'lat':('time',lat),'lon':('time',lon)}
                             ).dropna('time', how='all')
            Rho = xr.DataArray(Rho, dims=['time','pres'],
                              coords={'time':np.asarray(ds.JULD,dtype='datetime64[s]'),'pres':pnew,
                                     'lat':('time',lat),'lon':('time',lon)}
                              ).dropna('time', how='all')
            pMLD = xr.DataArray(pMLD, dims=['time'], 
                               coords={'time':np.asarray(ds.JULD,dtype='datetime64[s]'),
                                      'lat':('time',lat),'lon':('time',lon)}
                               ).dropna('time', how='all')

            rho_dropped = Rho.drop('lat').drop('lon').groupby('time').first()
            pML_dropped = pMLD.drop('lat').drop('lon').groupby('time').first()

            dt_intp = rho_dropped.time.diff('time')
            dt_intp = int(np.asarray(dt_intp.where(dt_intp>np.timedelta64(0)).min(),dtype=float)/(86400*1e9))
            if dt_intp == 0:
                dt_intp = 1
            tnew = np.arange(np.asarray(Rho.time,dtype='datetime64[D]')[0]+np.timedelta64(12,'h'), 
                            np.asarray(Rho.time,dtype='datetime64[D]')[-1]+np.timedelta64(24,'h'), 
                            np.timedelta64(dt_intp,'D'),
                            dtype='datetime64')
            pML_intp = pML_dropped.interp(time=tnew, method='linear')
            rho_intp = rho_dropped.interp(time=tnew, method='linear')

            dt = C.time.diff('time')
            dt = dt.where(dt>np.timedelta64(30,'D'))
            dtC = np.empty(0, dtype='datetime64')
            for i in range(len(dt)):
                if dt.data[i].astype(np.timedelta64) != np.timedelta64('NaT'):
                    dtC = np.append(dtC, np.asarray([dt.time.data[i-1], dt.time.data[i]], dtype='datetime64'))
            dt = Rho.time.diff('time')
            dt = dt.where(dt>np.timedelta64(30,'D'))
            dtR = np.empty(0, dtype='datetime64')
            for i in range(len(dt)):
                if dt.data[i].astype(np.timedelta64) != np.timedelta64('NaT'):
                    dtR = np.append(dtR, np.asarray([dt.time.data[i-1], dt.time.data[i]], dtype='datetime64'))

            if len(dtR)>0:
                if len(dtR)==2:
                    rho_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                elif len(dtR)==4:
                    rho_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                    rho_intp.sel(time=slice(dtR[2],dtR[3]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[2],dtR[3]))[:] = np.nan
                elif len(dtR)==6:
                    rho_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                    rho_intp.sel(time=slice(dtR[2],dtR[3]))[:] = np.nan
                    rho_intp.sel(time=slice(dtR[4],dtR[5]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[0],dtR[1]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[2],dtR[3]))[:] = np.nan
                    pML_intp.sel(time=slice(dtR[4],dtR[5]))[:] = np.nan
                else:
                    rho_intp[:] = np.nan
                    pML_intp[:] = np.nan

            CintgH = C[:,0].copy()
            CintgH[:] = np.nan
            ChlH = Chl[:,0].copy()
            ChlH[:] = np.nan
            ChlsML = ChlH.copy()

            i = 0
            for tt in C.time.data:
                if ~np.isnan(C.sel(time=tt)).all():
                    CintgH[i] = np.nansum(C.sel(time=tt)*dp)
                i += 1
                
            i = 0
            for tt in Chl.time.data:
                if ~np.isnan(Chl.sel(time=tt)).all():
                    ChlH[i] = np.nansum(Chl.sel(time=tt)*dp)
                    if tt in pML_dropped.time.data:
                        ChlsML[i] = np.nansum(Chl.sel(time=tt).where(Chl.pres<=10)*dp)/10*pML_dropped.sel(time=tt)
                i += 1

            i = 0
            for tt in CintgH.time.data:
                if tt in dtC:
                    if i % 2 != 0:
                        CintgH[i] = np.nan
                i += 1

            CintgH_tp = CintgH.interp(time=tnew, method='linear')
            ChlH_tp = ChlH.interp(time=tnew, method='linear')
            ChlsML_tp = ChlsML.interp(time=tnew, method='linear')
            nroll = int(30/dt_intp)
            if nroll % 2 == 0:
                nroll += 1
            min_ped = int(.5*nroll)
            if min_ped %2 == 0:
                min_ped += 1

            db = {'floatID':np.repeat(np.array([floatid]),len(tnew)), 'time':tnew, 
                 'Lat':CintgH_tp.lat.data, 'Lon':CintgH_tp.lon.data, 
                 'pML':pML_intp.data, 'pEu':pML_intp.data*np.nan,
                 'pML_rolled':pML_intp.rolling(time=nroll, center=True, min_periods=min_ped).mean().data,
                 'pEu_rolled':pML_intp.rolling(time=nroll, center=True, min_periods=min_ped).mean().data*np.nan,
                 'ChlH':ChlH_tp.data, 'ChlsML':ChlsML_tp.data, 
                 'CH':CintgH_tp.data,
                 'CH_rolled':CintgH_tp.rolling(time=nroll, center=True, min_periods=min_ped).mean().data,
                 'ChlH_rolled':ChlH_tp.rolling(time=nroll, center=True, min_periods=min_ped).mean().data,
                 'ChlsML_rolled':ChlsML_tp.rolling(time=nroll, center=True, min_periods=min_ped).mean().data}

            if v == 0:
                dfC = pd.DataFrame(data=db)
            else:
                dfC = dfC.append(pd.DataFrame(data=db))

            dHdt = np.asarray(pML_intp.rolling(time=nroll, center=True, 
                                               min_periods=min_ped).mean().diff('time'))/(dt_intp*86400)
            rD = dHdt * (.5*(np.asarray(pML_intp.rolling(time=nroll, center=True, 
                                                         min_periods=min_ped).mean().data[1:])
                            + np.asarray(pML_intp.rolling(time=nroll, center=True, 
                                                          min_periods=min_ped).mean().data[:-1])
                            ))**-1
            rD = xr.DataArray(rD, dims=['time'], 
                             coords={'time':(pML_intp.rolling(time=nroll, center=True, 
                                                             min_periods=min_ped).mean().time[:-1].astype('datetime64') 
                                            + np.asarray(.5*pML_intp.rolling(time=nroll, center=True, 
                                                                            min_periods=min_ped
                                                                            ).mean().time.diff('time').data))})

            pMLD_mid = xr.DataArray(.5*(np.asarray(pML_intp.data[1:]) + np.asarray(pML_intp.data[:-1])), dims=['time'],
                                   coords={'time':(pML_intp.time[:-1].astype('datetime64') 
                                                  + np.asarray(.5*pML_intp.time.diff('time').data))}
                                   ).rolling(time=nroll, center=True, min_periods=min_ped).mean()

            rPH = rD.copy()
            dCintgHdt = np.asarray(CintgH_tp.rolling(time=nroll, center=True, 
                                                    min_periods=min_ped).mean().diff('time'))/(dt_intp*86400)
            rPH[:] = (dCintgHdt * (.5*(np.asarray(CintgH_tp.rolling(time=nroll, center=True, 
                                                                   min_periods=min_ped).mean().data[1:])
                                      + np.asarray(CintgH_tp.rolling(time=nroll, center=True, 
                                                                    min_periods=min_ped).mean().data[:-1])
                                      ))**-1).data
            db = {'floatID':np.repeat(np.array([floatid]),len(rD)), 'time':rD.time.data,
                 'Lat':.5*(CintgH_tp.lat.data[1:]+CintgH_tp.lat.data[:-1]),
                 'Lon':.5*(CintgH_tp.lon.data[1:]+CintgH_tp.lon.data[:-1]),
                 'rD':rD.data, 'rPH':rPH.data}
            if v == 0:
                dfr = pd.DataFrame(data=db)
                v += 1
            else:
                dfr = dfr.append(pd.DataFrame(data=db))

/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater_equal
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater_equal
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:162: FutureWarning: In the future, NAT != NAT will be True rather than False.
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:168: FutureWarning: In the future, NAT != NAT will be True rather than False.
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater_equal
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater_equal
/home/takaya/miniconda3/envs/uptodate/lib/python3.6/site-package

In [142]:
# dfr.to_csv(op.join(ddir,'SOCCOM_rates.csv'), date_format='%Y-%m-%d %H:%M:%S')
# dfC.to_csv(op.join(ddir,'SOCCOM_Cs.csv'), date_format='%Y-%m-%d %H:%M:%S')